The TamilNadu state board school text books are available online for anyone to download.
https://www.tntextbooks.in/p/school-books.html

Here, we will be focusing on *10th standard Science subject* alone and this process is reproducible for any standard and subjects.

- https://www.tntextbooks.in/p/10th-books.html
- https://drive.google.com/file/d/1R4bFR_Qb8rFPPGKADPT6v2VE4P-CaWkF/view?usp=drivesdk

In [1]:
PREFACE_OFFSET = 8

CHAPTERS_PAGE_MAP = {
    "chapter_1": {"start": 1, "end": 15},
    "chapter_2": {"start": 16, "end": 31},
    "chapter_3": {"start": 32, "end": 41},
    "chapter_4": {"start": 42, "end": 58},
    "chapter_5": {"start": 59, "end": 73},
    "chapter_6": {"start": 74, "end": 90},
    "chapter_7": {"start": 91, "end": 105},
    "chapter_8": {"start": 106, "end": 123},
    "chapter_9": {"start": 124, "end": 136},
    "chapter_10": {"start": 137, "end": 154},
    "chapter_11": {"start": 155, "end": 172},
    "chapter_12": {"start": 173, "end": 186},
    "chapter_13": {"start": 187, "end": 199},
    "chapter_14": {"start": 200, "end": 217},
    "chapter_15": {"start": 218, "end": 228},
    "chapter_16": {"start": 229, "end": 242},
    "chapter_17": {"start": 243, "end": 260},
    "chapter_18": {"start": 261, "end": 273},
    "chapter_19": {"start": 274, "end": 285},
    "chapter_20": {"start": 286, "end": 299},
    "chapter_21": {"start": 300, "end": 314},
    "chapter_22": {"start": 315, "end": 328},
    "chapter_23": {"start": 329, "end": 333},
}

In [2]:
# from utils import CHAPTERS_PAGE_MAP, PREFACE_OFFSET
import fitz
import os

In [3]:
def extract_chapter_text(pdf_path, start_page, end_page, output_path):
    try:
        document = fitz.open(pdf_path)
        chapter_text = ""

        for page_num in range(start_page - 1, end_page):
            if page_num < len(document):
                page = document.load_page(page_num)
                chapter_text += page.get_text("text") + "\n\n"
            else:
                print(f"Warning: Page number {page_num + 1} is out of range.")

        with open(output_path, "w", encoding="utf-8") as fp:
            fp.write(chapter_text)

        print(f"Successfully extracted the chapter to '{output_path}")

    except Exception as e:
        print(f"An error occured: {e}")
        raise

In [4]:
PDF_FILE_PATH = "datasource/textbook.pdf"
OUTPUT_EXTRACT_PATH = "extracted"
os.makedirs(OUTPUT_EXTRACT_PATH, exist_ok=True)

In [5]:
def extract_raw_text_from_pdf(PDF_FILE_PATH, OUTPUT_EXTRACT_PATH):
    for chapter, page_details in CHAPTERS_PAGE_MAP.items():
        output_file = os.path.join(OUTPUT_EXTRACT_PATH, f"{chapter}.txt")
        chapter_start = PREFACE_OFFSET + page_details["start"]
        chapter_end = PREFACE_OFFSET + page_details["end"]
        extract_chapter_text(PDF_FILE_PATH, chapter_start, chapter_end, output_file)

In [6]:
extract_raw_text_from_pdf(PDF_FILE_PATH, OUTPUT_EXTRACT_PATH)

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax error: stitching function has no bounds

MuPDF error: syntax erro

In [ ]:
import os
import re
import json
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup


def clean_text(text):
    return re.sub(r"\s+", " ", text).strip()


def is_new_chapter_heading(text, tag_name):
    if tag_name not in ["h1", "h2"]:
        return False
    return re.match(r"^unit\s*-*\s*\d+", text.lower())


def is_topic_heading(text, tag_name):
    if not tag_name.startswith("h"):
        return False

    if re.match(r"^\d+(\.\d+)*", text):
        return True
    topic_patterns = [
        r"^(introduction|definition|properties|characteristics|types|examples)",
        r"^(method|procedure|process|technique|approach)",
        r"^(application|uses|importance|significance)",
        r"solved\s+problem",
        r"example\s*\d*",
    ]

    return any(re.search(pattern, text.lower()) for pattern in topic_patterns)


def should_skip_section(text):
    skip_keywords = [
        "table of contents",
        "learning objectives",
        "reference books",
        "glossary",
        "textbook evaluation",
        "points to remember",
        "ict corner",
        "practicals",
        "concept map",
        "index",
        "bibliography",
    ]
    return any(keyword in text.lower() for keyword in skip_keywords)


def should_skip_line(text):
    skip_patterns = [
        r"^figure\s*\d*:",
        r"^table\s*\d*:",
        r"^fig\s*\d*\.",
        r"^\s*\d+\s*$",
        r"^\s*[a-z]\)\s*$",
        r"^\s*\([a-z]\)\s*$",
    ]
    return any(re.match(pattern, text.lower()) for pattern in skip_patterns)


def save_chapter_data(chapter_data, output_dir):
    if not chapter_data:
        return

    for section_key in ["TOPIC WISE SECTIONS", "SOLVED PROBLEMS"]:
        if section_key in chapter_data:
            chapter_data[section_key] = [
                topic
                for topic in chapter_data[section_key]
                if topic.get("content") or topic.get("sub_topics")
            ]

            if not chapter_data[section_key]:
                del chapter_data[section_key]

    if not chapter_data.get("TOPIC WISE SECTIONS") and not chapter_data.get(
        "SOLVED PROBLEMS"
    ):
        print(
            f"Skipping empty chapter: {chapter_data.get('chapter_title', 'Unknown Title')}"
        )
        return

    chapter_number = chapter_data.get("chapter_number")
    output_path = os.path.join(output_dir, f"chapter_{chapter_number}.json")

    try:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(chapter_data, f, ensure_ascii=False, indent=4)
        print(f"Successfully saved: {output_path}")

        topic_count = len(chapter_data.get("TOPIC WISE SECTIONS", []))
        problem_count = len(chapter_data.get("SOLVED PROBLEMS", []))
        print(f"  - Topics: {topic_count}, Problems: {problem_count}")

    except IOError as e:
        print(f"Error writing file {output_path}: {e}")


def extract_and_structure_epub(epub_path, output_dir="structured_chapters"):
    try:
        book = epub.read_epub(epub_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"Created output directory: {output_dir}")
    except Exception as e:
        print(f"Fatal Error: Could not read EPUB file. Details: {e}")
        return

    toc_files = [
        item.get_name() for item in book.get_items_of_type(ebooklib.ITEM_NAVIGATION)
    ]
    print(f"Identified ToC files to skip: {toc_files}")

    chapter_count = 0
    current_chapter_data = None
    current_topic = None
    in_skipped_section = False
    awaiting_chapter_title_completion = False

    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        if item.get_name() in toc_files:
            continue

        soup = BeautifulSoup(item.get_content(), "html.parser")
        body = soup.find("body")
        if not body:
            continue

        elements = body.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p", "div", "li"])

        for elem in elements:
            text = elem.get_text(strip=True)
            if not text or should_skip_line(text):
                continue

            tag_name = elem.name
            is_heading = tag_name.startswith("h")

            print(f"Processing: [{tag_name}] {text[:100]}...")  # Debug output

            if is_heading and is_new_chapter_heading(text, tag_name):
                if current_chapter_data:
                    save_chapter_data(current_chapter_data, output_dir)

                chapter_count += 1
                current_chapter_data = {
                    "chapter_number": chapter_count,
                    "chapter_title": clean_text(text),
                    "TOPIC WISE SECTIONS": [],
                    "SOLVED PROBLEMS": [],
                }
                current_topic = None
                in_skipped_section = False
                awaiting_chapter_title_completion = True
                print(f"Started new chapter {chapter_count}: {clean_text(text)}")
                continue

            if (
                awaiting_chapter_title_completion
                and is_heading
                and not should_skip_section(text)
            ):
                current_chapter_data["chapter_title"] += f" - {clean_text(text)}"
                awaiting_chapter_title_completion = False
                print(
                    f"Completed chapter title: {current_chapter_data['chapter_title']}"
                )
                continue

            if not current_chapter_data:
                continue

            if is_heading and should_skip_section(text):
                in_skipped_section = True
                current_topic = None
                print(f"Entering skipped section: {text}")
                continue

            if is_heading and is_topic_heading(text, tag_name):
                in_skipped_section = False
                awaiting_chapter_title_completion = False

                if "solved" in text.lower() and "problem" in text.lower():
                    section_key = "SOLVED PROBLEMS"
                else:
                    section_key = "TOPIC WISE SECTIONS"

                topic_title = clean_text(text)
                new_topic = {"topic_title": topic_title, "content": []}

                topic_num_match = re.match(r"^(\d+(?:\.\d+)*)", topic_title)
                if topic_num_match:
                    new_topic["topic_number"] = topic_num_match.group(1)

                current_chapter_data[section_key].append(new_topic)
                current_topic = new_topic
                print(f"Created new topic: {topic_title}")
                continue

            if not is_heading and not in_skipped_section and current_topic is not None:
                awaiting_chapter_title_completion = False
                content_text = clean_text(text)
                if content_text and len(content_text) > 10:
                    current_topic["content"].append(content_text)
                    print(f"Added content: {content_text[:50]}...")

    if current_chapter_data:
        save_chapter_data(current_chapter_data, output_dir)

    print(f"\nExtraction complete. Processed {chapter_count} chapters.")


epub_file_path = "datasource/textbook.epub"
output_directory = "structured_chapters"
extract_and_structure_epub(epub_file_path, output_directory)

Created output directory: structured_chapters
Identified ToC files to skip: []
Processing: [p] Table Of Contents...
Processing: [p] Unit...
Processing: [p] Title...
Processing: [p] Page No....
Processing: [p] Laws of Motion...
Processing: [p] Optics...
Processing: [p] Thermal Physics...
Processing: [p] Electricity...
Processing: [p] Acoustics...
Processing: [p] Nuclear Physics...
Processing: [p] Atoms and Molecules...
Processing: [p] Periodic Classification of Elements...
Processing: [p] Solutions...
Processing: [p] Types of Chemical Reactions...
Processing: [p] Carbon and its Compounds...
Processing: [p] Unit...
Processing: [p] Title...
Processing: [p] Page No....
Processing: [p] Plant Anatomy and Plant Physiology...
Processing: [p] Structural Organisation of Animals...
Processing: [p] Transportation in Plants and Circulation in Animals...
Processing: [p] Nervous System...
Processing: [p] Plant and Animal Hormones...
Processing: [p] Reproduction in Plants and Animals...
Processing: [p